In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_regression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from docx import Document

from statsmodels.iolib.summary2 import summary_col
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from xgboost import XGBClassifier, XGBRegressor
import pandas as pd
import numpy as np

from linearmodels.panel import PanelOLS
import statsmodels.formula.api as smf
import statsmodels.api as sm


import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('float_format', '{:f}'.format)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.ticker as mtick

In [2]:
import scipy.stats as st
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

In [3]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Model database : 1,5 MM - 30% do publico de interesse

In [4]:
# Import final data base de inscritos presentes em todas as provas sem missing ~32 MM
df = pd.read_csv('df_presentes_sm_marcada.csv', sep = ';')

print(df.shape)


(32601016, 56)


# Fixed Effects Dataset 

In [5]:
df_efixo = df.copy()

In [6]:
df_efixo['sóI'] = df_efixo['I'] * (1 - df_efixo['C'])
df_efixo['sóC'] = df_efixo['C'] * (1 - df_efixo['I'])
df_efixo['IeC'] = df_efixo['I'] * df_efixo['C']

In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
NU_ANO,32601016.000000,2018.201370,2.554162,2015.000000,2016.000000,2018.000000,2020.000000,2023.000000
NU_IDADE,32601016.000000,21.232734,7.002890,10.000000,17.000000,19.000000,22.000000,92.000000
TP_ESTADO_CIVIL,32601016.000000,0.496562,0.587578,0.000000,0.000000,0.000000,1.000000,4.000000
TP_COR_RACA,32601016.000000,2.089940,1.017052,0.000000,1.000000,2.000000,3.000000,6.000000
TP_ST_CONCLUSAO,32601016.000000,1.676778,0.790806,1.000000,1.000000,1.000000,2.000000,4.000000
TP_ANO_CONCLUIU,32601016.000000,2.385633,3.640825,0.000000,0.000000,0.000000,3.000000,17.000000
TP_ESCOLA,32601016.000000,1.383183,0.626231,1.000000,1.000000,1.000000,2.000000,4.000000
NU_NOTA_CN,32601016.000000,489.187838,76.268570,0.000000,431.400000,480.500000,540.200000,885.600000
NU_NOTA_CH,32601016.000000,534.996790,83.183849,0.000000,477.000000,540.300000,595.300000,868.300000
NU_NOTA_LC,32601016.000000,517.479742,71.242444,0.000000,471.300000,522.100000,567.800000,826.100000


## Criando as variáveis dummys específicas

In [7]:
# feature transform
df_efixo['RACA'] = np.where(df_efixo.TP_COR_RACA == 1, 'Branca',
                                              np.where(df_efixo.TP_COR_RACA == 3, 'Parda','Raca_outra'))

# variable
unit_col_name = 'RACA'

#Create the dummy variables, one for each category
df_dummies = pd.get_dummies(df_efixo[unit_col_name])

df_efixo_with_dummies = df_efixo.join(df_dummies)

In [8]:
#df_efixo_with_dummies['RACA'].value_counts()

In [9]:
#df_efixo_with_dummies['TP_COR_RACA'].value_counts()

In [10]:
# feature transform
df_efixo['fx_renda'] = np.where(df_efixo.cat_Q006 == 0, 'renda_0e1',
                                                np.where(df_efixo.cat_Q006 == 1, 'renda_0e1', 'renda_15'))

# variable
unit_col_name = 'fx_renda'


#Create the dummy variables, one for each category
df_dummies = pd.get_dummies(df_efixo[unit_col_name])

df_efixo_with_dummies = df_efixo_with_dummies.join(df_dummies)

In [15]:
df_efixo_with_dummies.shape

(32601016, 62)

In [16]:
#df_efixo_with_dummies['REG_UF'].value_counts()

In [11]:
# variable
unit_col_name = 'REG_UF'

#Create the dummy variables, one for each category
df_dummies = pd.get_dummies(df_efixo[unit_col_name])

df_efixo_with_dummies = df_efixo_with_dummies.join(df_dummies)

In [18]:
#df_efixo_with_dummies.head(10)

In [19]:
#df_efixo_with_dummies['TP_ESCOLA'].value_counts()

In [12]:
# feature transform
df_efixo['ESCOLA'] = np.where(df_efixo.TP_ESCOLA == 2, 'escpubl','escpartic_outra')

# variable
unit_col_name = 'ESCOLA'


#Create the dummy variables, one for each category
df_dummies = pd.get_dummies(df_efixo[unit_col_name])

df_efixo_with_dummies = df_efixo_with_dummies.join(df_dummies)

In [21]:
df_efixo_with_dummies.shape

(32601016, 69)

In [22]:
#H	0Não sei.------------------------------------------------------------------------------>nao sabe ou nunca0
#A	1Nunca estudou.------------------------------------------------------------------------>nao sabe ou nunca0
#B	2Não completou a 4ª série/5º ano do Ensino Fundamental.--------------------------------->P_fund_incompl1
#C	3Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental--->P_fund_incompl1

#D	4Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio------>P_Ens_fundamental2
#E	5Completou o Ensino Médio, mas não completou a Faculdade------------------------------>P_Ens_medio3

#F	6Completou a Faculdade, mas não completou a Pós-graduação-----------------------------> Paigradmais4
#G	7Completou a Pós-graduação--------------------------------------------------------> Paigradmais4


In [13]:
# feature transform
#Q002 - estudo da mae
#A	1Nunca estudou.-------------------------------------------------------------------------->escMaenuncaincompl2
#B	2Não completou a 4ª série/5º ano do Ensino Fundamental.---------------------------------->escMaenuncaincompl2
#C	3Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental--->escMaenuncaincompl2
#D	4Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio------>escMaefund3
#E	5Completou o Ensino Médio, mas não completou a Faculdade---------------------------------->escMaemed4
#F	6Completou a Faculdade, mas não completou a Pós-graduação--------------------------------->escMaegradmais5
#G	7Completou a Pós-graduação----------------------------------------------------------------->escMaegradmais5
#H	0Não sei.--------------------------------------------------------------------------------->escMaedesc1
#'A' : 1,
#'B' : 2,
#'C' : 3,
#'D' : 4,
#'E' : 5,
#'F' : 6,
#'G' : 7,
#'H' : 0,
#-1  : -1    

df_efixo['ESCOLA_MAE'] = np.where(df_efixo.cat_Q002 == 0, 'Mae_desc_nada0',
                                     np.where(df_efixo.cat_Q002 == 1, 'Mae_desc_nada0',
                                              np.where(df_efixo.cat_Q002 == 2, 'Mae_fund_incompl1',
                                                       np.where(df_efixo.cat_Q002 == 3, 'Mae_fund_incompl1',
                                                                np.where(df_efixo.cat_Q002 == 4, 'Mae_Ens_fundamental2',
                                                                         np.where(df_efixo.cat_Q002 == 5, 'Mae_Ens_medio3','Mae_ESup_mais'))))))

# variable
unit_col_name = 'ESCOLA_MAE'

#Create the dummy variables, one for each category
df_dummies = pd.get_dummies(df_efixo[unit_col_name])

df_efixo_with_dummies = df_efixo_with_dummies.join(df_dummies)

In [14]:
# feature transform
#Q001 - estudo da pai
#A	1Nunca estudou.-------------------------------------------------------------------------->escPainuncaincompl2
#B	2Não completou a 4ª série/5º ano do Ensino Fundamental.---------------------------------->escPainuncaincompl2
#C	3Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental--->escPainuncaincompl2
#D	4Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio------>escPaifund3
#E	5Completou o Ensino Médio, mas não completou a Faculdade---------------------------------->escPaimed4
#F	6Completou a Faculdade, mas não completou a Pós-graduação--------------------------------->escPaigradmais5
#G	7Completou a Pós-graduação----------------------------------------------------------------->escPaigradmais5
#H	0Não sei.--------------------------------------------------------------------------------->escPaidesc1
#'A' : 1,
#'B' : 2,
#'C' : 3,
#'D' : 4,
#'E' : 5,
#'F' : 6,
#'G' : 7,
#'H' : 0,
#-1  : -1    


df_efixo['ESCOLA_PAI'] = np.where(df_efixo.cat_Q001 == 0, 'Pai_desc_nada0',
                                     np.where(df_efixo.cat_Q001 == 1, 'Pai_desc_nada0',
                                              np.where(df_efixo.cat_Q001 == 2, 'Pai_fund_incompl1',
                                                       np.where(df_efixo.cat_Q001 == 3, 'Pai_fund_incompl1',
                                                                np.where(df_efixo.cat_Q001 == 4, 'Pai_Ens_fundamental2',
                                                                         np.where(df_efixo.cat_Q001 == 5, 'Pai_Ens_medio3','Pai_ESup_mais'))))))

# variable
unit_col_name = 'ESCOLA_PAI'

#Create the dummy variables, one for each category
df_dummies = pd.get_dummies(df_efixo[unit_col_name])

df_efixo_with_dummies = df_efixo_with_dummies.join(df_dummies)

In [ ]:
df_efixo_with_dummies['cat_Q001'].value_counts()

In [26]:
df_efixo['ESCOLA_PAI'].value_counts()

Pai_fund_incompl1       11046733
Pai_Ens_medio3           8854702
Pai_ESup_mais            4414004
Pai_desc_nada0           4382711
Pai_Ens_fundamental2     3902866
Name: ESCOLA_PAI, dtype: int64

In [16]:
# feature transform
df_efixo['TAM_FAMILIA'] = np.where(df_efixo.Q005 <= 3, 'tamfam3peq',
                                            np.where(df_efixo.Q005 <= 5, 'tamfam5med','tamfam6gde'))

# variable
unit_col_name = 'TAM_FAMILIA'

#Create the dummy variables, one for each category
df_dummies = pd.get_dummies(df_efixo[unit_col_name])

#Junção das Variáveis Dummy ao DataFrame Original
df_efixo_with_dummies = df_efixo_with_dummies.join(df_dummies)

In [17]:
#Q007	DOMESTICO = 1 OU MAIS
#Q008	BANHEIRO = 2 OU MAIS
#Q009	QUARTOS = 3 OU MAIS
#Q010	CARRO = 1 OU MAIS
#Q011	MOTO = 1 OU MAIS
#Q012	GELADEIRA = 1 OU MAIS
#Q014	MAQ DE LAVAR = 1 OU MAIS
#Q016	MICROONDAS = 1 OU MAIS
#Q021	TV_ASSINATURA = 1 OU MAIS
#Q022	CELULAR = 1 OU MAIS
#Q023	TELEFONE_FIXO = 1 OU MAIS
#-1  : -1,
#'A' : 0,
#'B' : 1,
#'C' : 2,
#'D' : 3,
#'E' : 4  

df_efixo['banheiros2'] = np.where(df_efixo.cat_Q008 >=2, 1, 0)
df_efixo['banheiros1'] = np.where(df_efixo.cat_Q008 >=1, 1, 0)
df_efixo['quartos2'] = np.where(df_efixo.cat_Q009 >=2, 1, 0)
#df_efixo['carro'] = np.where(df_efixo.cat_Q010 >=1, 1, 0)#naoeh basico
df_efixo['geladeira'] = np.where(df_efixo.cat_Q012 >=1, 1, 0)
df_efixo['maqlavar'] = np.where(df_efixo.cat_Q014 >=1, 1, 0)
df_efixo['microondas'] = np.where(df_efixo.cat_Q016 >=1, 1, 0)
#df_efixo['TVassinatura'] = np.where(df_efixo.cat_Q021 >=1, 1, 0)#naoeh basico
#df_efixo['telfixo'] = np.where(df_efixo.cat_Q023 >=1, 1, 0)#naoeh basico

In [18]:
df_efixo['basicocompleto'] = df_efixo['banheiros1'] + df_efixo['quartos2']+ df_efixo['geladeira'] + df_efixo['maqlavar'] + df_efixo['microondas']

In [30]:
#df_efixo['basicocompleto'].value_counts()

In [19]:
# feature transform
#basicocompleto -> ter necessariamente 5 coisas:  dois banheiros, dois quartos, geladeira, maqlavar, microondas
df_efixo['infra_domicilio'] = np.where(df_efixo.basicocompleto == 5, 'infra_basicacompleta','infra_basicaincompleta')

In [20]:
# variable
unit_col_name = 'infra_domicilio'

#Create the dummy variables, one for each category
df_dummies = pd.get_dummies(df_efixo[unit_col_name])

df_efixo_with_dummies = df_efixo_with_dummies.join(df_dummies)

In [21]:
df_efixo_with_dummies.rename(columns={'Centro-oeste': 'CentroOeste'}, inplace=True)

In [22]:
df_efixo_with_dummies['NU_MEDIA'] = (df_efixo_with_dummies['NU_NOTA_MT'] + df_efixo_with_dummies['NU_NOTA_CN'] + df_efixo_with_dummies['NU_NOTA_CH'] + df_efixo_with_dummies['NU_NOTA_LC']) / 4

In [24]:
df=df_efixo_with_dummies

In [29]:
keep_vars = [
    
    'I', 'C', 'sóI', 'sóC', 'IeC', 'NU_MEDIA','NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 
    'NU_NOTA_REDACAO', 'FEMININO','Branca', 'Parda', 'Raca_outra', 'renda_0e1', 'renda_15', 'CentroOeste', 'Nordeste',
    'Norte',    'Sudeste', 'Sul', 'escpartic_outra', 'escpubl', 'Mae_ESup_mais', 'Mae_Ens_fundamental2', 
    'Mae_Ens_medio3', 'Mae_desc_nada0', 'Mae_fund_incompl1', 'Pai_ESup_mais', 'Pai_Ens_fundamental2', 
    'Pai_Ens_medio3', 'Pai_desc_nada0', 'Pai_fund_incompl1', 'tamfam3peq', 'tamfam5med', 'tamfam6gde',
    'infra_basicacompleta','infra_basicaincompleta'
] 

In [30]:
df=df_efixo_with_dummies[keep_vars]

In [42]:
# Lista das variáveis de interesse
variaveis_de_interesse = [
    'I', 'C', 'sóI', 'sóC', 'IeC', 'NU_MEDIA','NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 
    'NU_NOTA_REDACAO', 'FEMININO','Branca', 'Parda', 'Raca_outra', 'renda_0e1', 'renda_15', 'CentroOeste', 'Nordeste',
    'Norte',    'Sudeste', 'Sul', 'escpartic_outra', 'escpubl', 'Mae_ESup_mais', 'Mae_Ens_fundamental2', 
    'Mae_Ens_medio3', 'Mae_desc_nada0', 'Mae_fund_incompl1', 'Pai_ESup_mais', 'Pai_Ens_fundamental2', 
    'Pai_Ens_medio3', 'Pai_desc_nada0', 'Pai_fund_incompl1', 'tamfam3peq', 'tamfam5med', 'tamfam6gde',
    'infra_basicacompleta','infra_basicaincompleta'
]

# Agrupando por ano e calculando as estatísticas descritivas para as variáveis selecionadas
descricao_por_ano = df.groupby('NU_ANO')[variaveis_de_interesse].agg(['count', 'mean', 'std', 'min', 'median', 'max'])

# Renomear colunas para um formato mais legível (opcional)
descricao_por_ano.columns = ['_'.join(col).strip() for col in descricao_por_ano.columns.values]

# Visualizando a descrição por ano
print(descricao_por_ano)

# Se quiser salvar em um arquivo Excel ou CSV
descricao_por_ano.to_excel('descricao_por_ano_presentes.xlsx')

        I_count   I_mean    I_std  I_min  I_median  I_max  C_count   C_mean  \
NU_ANO                                                                        
2015    5411032 0.657191 0.474648      0  1.000000      1  5411032 0.691183   
2016    5613987 0.665103 0.471954      0  1.000000      1  5613987 0.642412   
2017    4283552 0.708652 0.454383      0  1.000000      1  4283552 0.625665   
2018    3768001 0.758048 0.428265      0  1.000000      1  3768001 0.602783   
2019    3701947 0.788770 0.408181      0  1.000000      1  3701947 0.561707   
2020    2561304 0.839559 0.367014      0  1.000000      1  2561304 0.557758   
2021    2238106 0.915390 0.278301      0  1.000000      1  2238106 0.627754   
2022    2344823 0.919588 0.271931      0  1.000000      1  2344823 0.578413   
2023    2678264 0.916954 0.275952      0  1.000000      1  2678264 0.520679   

          C_std  C_min  C_median  C_max  sóI_count  sóI_mean  sóI_std  \
NU_ANO                                                   

In [ ]:
df_efixo_with_dummies.to_csv('presentescom_categoricas.csv', sep = ';', index = False)

In [ ]:
# Lista das variáveis de interesse
variaveis_de_interesse = [
    'I', 'C', 'sóI', 'sóC', 'IeC', 'NU_MEDIA','NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 
    'NU_NOTA_REDACAO', 'FEMININO','Branca', 'Parda', 'Raca_outra', 'renda_0e1', 'renda_15', 'CentroOeste', 'Nordeste',
    'Norte',    'Sudeste', 'Sul', 'escpartic_outra', 'escpubl', 'Mae_ESup_mais', 'Mae_Ens_fundamental2', 
    'Mae_Ens_medio3', 'Mae_desc_nada0', 'Mae_fund_incompl1', 'Pai_ESup_mais', 'Pai_Ens_fundamental2', 
    'Pai_Ens_medio3', 'Pai_desc_nada0', 'Pai_fund_incompl1', 'tamfam3peq', 'tamfam5med', 'tamfam6gde',
    'infra_basicacompleta','infra_basicaincompleta'
]

# Agrupando por ano e calculando as estatísticas descritivas para as variáveis selecionadas
descricao = df[variaveis_de_interesse].agg(['count', 'mean', 'std', 'min', 'median', 'max'])

# Renomear colunas para um formato mais legível (opcional)
descricao.columns = ['_'.join(col).strip() for col in descricao.columns.values]

# Visualizando a descrição
print(descricao)

# Se quiser salvar em um arquivo Excel ou CSV
descricao.to_excel('descricao_presentes_TOTAL.xlsx')

## verificar diferença entre médias dos alunos do 3ano

In [8]:
df['NU_MEDIA'] = (df['NU_NOTA_MT'] + df['NU_NOTA_CN'] + df['NU_NOTA_CH'] + df['NU_NOTA_LC']) / 4

In [9]:

# Agrupando por ano e calculando a média de NU_MEDIA apenas para os concluintes do 3º ano
media_por_ano = df.groupby('NU_ANO').apply(lambda x: x.loc[x['concluinte_3ano'] == 1, 'NU_MEDIA'].mean()).reset_index()

# Renomeando as colunas para maior clareza
media_por_ano.columns = ['Ano', 'Media_NU_MEDIA']

# Exibindo o resultado
print(media_por_ano)

    Ano  Media_NU_MEDIA
0  2015      502.268109
1  2016      506.260637
2  2017      511.614266
3  2018      527.662658
4  2019      505.836556
5  2020      512.182619
6  2021      506.269883
7  2022      515.787097
8  2023      513.570647


In [10]:

# Agrupando por ano e calculando a média de NU_MEDIA apenas para os concluintes do 3º ano
media_por_ano = df.groupby('NU_ANO').apply(lambda x: x.loc[x['concluinte_3ano'] == 0, 'NU_MEDIA'].mean()).reset_index()

# Renomeando as colunas para maior clareza
media_por_ano.columns = ['Ano', 'Media_NU_MEDIA']

# Exibindo o resultado
print(media_por_ano)

    Ano  Media_NU_MEDIA
0  2015      503.590743
1  2016      506.337196
2  2017      517.718798
3  2018      534.424054
4  2019      509.564758
5  2020      513.178631
6  2021      518.370417
7  2022      526.489868
8  2023      522.687436
